In [190]:
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import statistics
from collections import Counter

In [58]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'

In [90]:
df = pd.read_excel(file)

In [228]:
liste = df['textes'].to_list()

In [61]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

pie = []

sentences: List[str] = liste
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.lasla.imports import get_iterator_and_processor
for sentence_group in sentences:
    iterator, processor = get_iterator_and_processor()
    pie.append(tagger.tag_str(sentence_group, iterator=iterator, processor=processor) )

In [67]:
os.chdir('/home/noe/Documents/2022-2023/ENC/Mémoire/pdv_lemmes/')

In [68]:
while i < len(pie):
    
    pos = []
    lemma = []
    
    for j in pie[i]:
        
        lemma.append(pie[i][j]['lemma'])
        with open(f'pdv_lemma{i}.txt','w') as file:
            file.write(''.join(lemma))

TypeError: list indices must be integers or slices, not dict

In [75]:
def get_lemma_lists(list_of_lists):
    result = []
    for sublist in list_of_lists:
        lemma_list = []
        for dictionary in sublist:
            if 'lemma' in dictionary:
                lemma_list.append(dictionary['lemma'])
        result.append(lemma_list)
    return result

In [229]:
x = get_lemma_lists(pie)

In [249]:
lemma

,lemma
0,colligo3 pontifex et2 Phariseus consilium in u...
1,ille felix describo antiquitas qui1 ex alienus...
2,etsi2 causa noster iustitia uulgaris fama prel...
3,ne4 per excogito malignitas astutia iste Roman...
4,uir industrius et2 illustris Romanus comes Tho...
...,...
358,fauorabilis petitio supplico effectus1 debeo p...
359,tenor preso notus2 facio uniuersi quod2 cum3 m...
360,ad2 hic1 superus dispensatio consilium princip...
361,consuesco innascor benignitas consuetudo regno...


In [231]:
x = [' '.join(i) for i in x]

In [233]:
lemma = pd.DataFrame(x)

In [244]:
def clean_text(text):
    char  = [';',':','!','«','»',',','“','”','.','?','[',']']
    pattern = '[' + ''.join(char) + ']'
    new_str = re.sub(pattern,'', text)
    lol = re.sub(r' +',' ',new_str)
    return lol


In [246]:
def remove_chars(input_str):
    chars_to_remove = [';', ':', '!', '«', '»', ',', '“', '”', '.', '?', '[', ']']
    for char in chars_to_remove:
        input_str = input_str.replace(char, '')
    lol = re.sub(r' +',' ',input_str)
    return lol


In [235]:
lemma = lemma.rename(columns={0:'lemma'})

In [248]:
lemma['lemma'] = lemma['lemma'].apply(remove_chars)

In [250]:
def livre_id(string):
    new_str = re.sub(r'[A-Za-z]','', string)
    return new_str

In [251]:
df['ID'] = ''

In [252]:
for i in range(len(df)):
    df['ID'][i] = str(df['Livre'][i])+'_'+ str(df['Numéro de la charte'][i])

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [253]:
ids = df['ID'].to_list()

In [254]:
lemma['ID'] = ids

In [255]:
lemma = lemma[['ID','lemma']]

In [256]:
lemma['ID'] = lemma['ID'].apply(livre_id)

In [261]:
test = []
for i in range(len(lemma)):
    test.append(len(lemma['lemma'][i].split()))

In [274]:
statistics.median(test)

163

In [265]:
filtre = lemma[lemma['lemma'].str.split().apply(len) > 1000]
filtre = filtre.reset_index()

In [266]:
filtre

,index,ID,lemma
0,0,1_1,colligo3 pontifex et2 Phariseus consilium in u...
1,2,1_3,etsi2 causa noster iustitia uulgaris fama prel...
2,20,1_21,leuo2 in circuitus oculus uester arrigo filius...
3,27,1_28,etsi1 tam iustus quam1 uehemens causa dolor et...
4,92,3_1,dum2 debitor noster cirografus lego2 dum2 kale...
5,339,6_9,regno nos feliciter et2 triumpho ac1 uictorios...


In [185]:
def thousand(string):
    final = string[:1000]
    return final

In [186]:
filtre = filtre['lemma'].apply(thousand).to_frame()

In [225]:
x = lemma['lemma'].apply(clean_text)

In [227]:
x

0      colligo3 pontifex et2 Phariseus consilium in u...
1      ille felix describo antiquitas , qui1 ex alien...
2      etsi2 causa noster iustitia , uulgaris fama pr...
3      ne4 per excogito malignitas astutia iste Roman...
4      uir industrius et2 illustris Romanus , comes T...
                             ...                        
358    fauorabilis petitio supplico effectus1 debeo p...
359    tenor preso notus2 facio uniuersi , quod2 , cu...
360    ad2 hic1 superus dispensatio consilium princip...
361    consuesco innascor benignitas , consuetudo reg...
362    notus2 facio uniuersitas uester , qui1 . . . d...
Name: lemma, Length: 363, dtype: object